# Importando Bibliotecas e módulos necessários

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opendssdirect as dss
from powerflow.bus import bus_dictionary
from powerflow.batery import distbess_dictionary
from powerflow.generators import distgen_dictionary,add_gd
from powerflow.get_powers import get_dist_power,get_bus_power
from powerflow.load import add_LoadToBus,add_load
from powerflow.text_comands import set_frequency,new_circuit,line_geometry,add_line,add_infos


# Diretórios dos dados


In [2]:
# Diretório de datasets
dir_datasets = 'C:\\Users\\joao9\\GitHub\\opendss\\datasets\\'

#Diretório de arquivos .dss
dir_dss = dir_datasets + 'arquivos_dss\\'

# Diretório de arquivos de geração PV
dir_pv = 'datasets\\generation_profiles\\'

# Diretório de arquivos de cargas
dir_cargas = 'datasets\\load_profiles\\'


# Definições iniciais do circuito

In [3]:

Text = dss.Text
Circuit = dss.Circuit
Solution = dss.Solution

# Definições da MatPlotLib
plt.close('all')
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (8,4)
plt.rcParams["figure.dpi"] = 100
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.sans-serif'] = ['cmr10']
plt.rcParams['axes.unicode_minus'] = False

#Potências máximas
PowerLoad=4
PowerPV=6 #Potência total dos módulos

# Definições do BESS
Pmin=0.320
BESS_P_MAX = 5
BESS_C_MAX = 12
SOC_MIN = 0.1 # 10%
SOC_MAX = 0.9 # 90%
BESS_E_MIN = BESS_C_MAX * SOC_MIN
BESS_E_MAX = BESS_C_MAX * SOC_MAX

# Definições do intervalo de simulação
# nconsumidores = len(os.listdir(dir_cargas))
nconsumidores = 3
nbarramentos = 4
horas = 24 #hours
intervalo = 30 #min

# Criação do modelo base do arquivo.dss

In [4]:
with open(dir_dss + 'ModelagemTeste.dss', 'w') as file:
    file.write(set_frequency(60) + '\n')
    file.write(new_circuit('ModelagemTeste', str(1).zfill(3), 0.22) + '\n')
    file.write(line_geometry(0.52,0.63,'km',3.338,3,3,[-120.00,0.00,60.00],[1000.00,1000.00,1000.00])+ '\n')
    for i in range(1,nbarramentos,1):
        file.write(add_line(i,str(i).zfill(3),str(i+1).zfill(3),25,'m',3)+ '\n')
    
    file.write(add_infos())

opendssmodel = dir_dss + 'ModelagemTeste.dss'

# Criação dos dicionários que irão armazenar as inforamções das cargas, geraçãoPV e bateria

In [5]:
profiles = os.listdir(dir_cargas)

buses = bus_dictionary(nbarramentos)
for i in range(3):
    bus = 'bus_'+str(i+1).zfill(3)
    name = profiles[i].replace('.csv','')
    add_LoadToBus(buses,bus,name,2,0.22,1000,dir_cargas+profiles[i])

display(buses)

{'bus_001': {'MAC003982': {'phases': 2,
   'voltage': 0.22,
   'power': 1000,
   'profile':                           datetime  Ppower    Qpower
   0      2012-07-06 12:00:00.0000000   0.194  0.000194
   1      2012-07-06 12:30:00.0000000   0.159  0.000159
   2      2012-07-06 13:00:00.0000000   0.161  0.000161
   3      2012-07-06 13:30:00.0000000   0.188  0.000188
   4      2012-07-06 14:00:00.0000000   0.144  0.000144
   ...                            ...     ...       ...
   28818  2014-02-27 22:30:00.0000000   0.123  0.000123
   28819  2014-02-27 23:00:00.0000000   0.133  0.000133
   28820  2014-02-27 23:30:00.0000000   0.146  0.000146
   28821  2014-02-28 00:00:00.0000000   0.193  0.000193
   28822  2014-02-28 00:00:00.0000000   0.193  0.000193
   
   [28823 rows x 3 columns]}},
 'bus_002': {'MAC003983': {'phases': 2,
   'voltage': 0.22,
   'power': 1000,
   'profile':                           datetime  Ppower    Qpower
   0      2012-07-06 12:00:00.0000000   0.098  0.000098
   

# Garantindo que o Opendss esteja limpo

In [6]:
dss.Basic.ClearAll()
dss.Basic.Start(0)
Text.Command("Clear")
Text.Command(f"Compile {opendssmodel}")

print(Circuit.AllBusNames)

<bound method ICircuit.AllBusNames of <opendssdirect.Circuit.ICircuit object at 0x000001EE36484150>>


# Fluxo de carga

In [7]:
column_power = ['datetime', 'line', 'bus_1', 'bus_2', 'P', 'Q']
power_df = pd.DataFrame(columns = column_power)

# column_node_power = ['datetime', 'bus','V', 'P', 'Q']
# bus_power = pd.DataFrame(columns = column_power)

column_node_power = ['datetime', 'bus', 'P', 'Q', 'V_a_b_c']
node_power_df1 = pd.DataFrame(columns=column_node_power)



range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
for time in range_time:

  dss.Basic.ClearAll()
  dss.Basic.Start(0)
  Text.Command("Clear")
  Text.Command(f"Compile {opendssmodel}")

  #Acessa os barramentos do dicionário
  for bus in buses.keys():
    loads = buses[bus].keys()
    for load in loads:
      carga = buses[bus][load]
      new_command = add_load(time,load,bus, carga['phases'],carga['voltage'], carga['power'], carga['profile'])
      Text.Command(new_command)

  # for gen in distgen:
  #   new_command = new_command + '\n' + add_gd(time, gen, distgen[gen]['bus'], distgen[gen]['phases'], 
  #   distgen[gen]['conn'], distgen[gen]['voltage'], distgen[gen]['power'], distgen[gen]['profile'])
 
  Text.Command("Set VoltageBases = [.22]")
  Text.Command("CalcVoltageBases")
  Text.Command("Set maxiterations = 500")
  Text.Command('set mode = snap')
  Text.Command("Solve")

  # Power Flow to DataFrame
  power = np.array(dss.PDElements.AllPowers())
  # print(power)
  power = power.reshape((-1, 6))
  # print(power)
  power = pd.DataFrame(power, columns=['P_a', 'Q_a', 'P_b', 'Q_b', 'P_c', 'Q_c'])
  

  # Create new column that contains sum of specific columns
  power['P'] = power[['P_a', 'P_b', 'P_c']].sum(axis=1)
  power['Q'] = power[['Q_a', 'Q_b', 'Q_c']].sum(axis=1)
  # display(power)
  
  # Get the index for each row of the Data Frame
  power_flow_index = []
  for line in dss.Lines:
      power_flow_index.append([time, line.Name(), line.Bus1(), line.Bus2()])
      power_flow_index.append([time, line.Name(), line.Bus1(), line.Bus2()])
  # print(power_flow_index)

  power[['datetime', 'line', 'bus_1', 'bus_2']] = power_flow_index
  # display(power)
  power_df = pd.concat([power_df, power])
  # display(power_df)


  # Get distributed generation power
  node_power = get_dist_power(time,column_node_power,power,buses,dss)
  # display(node_power)


  # Add Grid and Losses power to DataFrame
  # The last "Power Balance" of the system, to be used in the smoothing algorithm.
  node_power = pd.concat([node_power, pd.DataFrame([[time, 'grid', -dss.Circuit.TotalPower()[0], -dss.Circuit.TotalPower()[1], 0]], columns=column_node_power)])
  node_power = pd.concat([node_power, pd.DataFrame([[time, 'losses', dss.Circuit.Losses()[0], dss.Circuit.Losses()[1], 0]], columns=column_node_power)])


  node_power_df1 = pd.concat([node_power_df1, node_power])

  

0 128.7819965915692
-121.68958423512392 30.3258262021987
-29.885502365629968 8.590508667033459e-14
-1.1914234658304662e-13 0
0 117.6033711453934
-111.6934419935661 30.65070297564995
-30.20580274448079 2.304569785504386e-13
-2.158990456600733e-13 0


C:\Users\joao9\AppData\Local\Temp\ipykernel_14876\1026979376.py:12: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_14876\1026979376.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  range_time = pd.date_range('2012-07-06 12:00:00', periods=horas*60/intervalo, freq=str(intervalo)+'T')
C:\Users\joao9\AppData\Local\Temp\ipykernel_14876\1026979376.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  power_df = pd.concat([pow

0 83.84779290856955
-80.84920943671936 28.79704790721949
-28.416616182014476 4.5761325949374694e-14
5.2403005819300515e-14 0
0 78.82200340485664
-76.1721363393951 33.66998467296645
-33.15216816756903 3.3269985418312334e-13
-1.4606850705159728e-13 0
0 45.99852581852733
-45.09722570251636 28.435215955669484
-28.076844295642218 -7.397935651379295e-14
-1.0457647526531271e-13 0
0 141.56986747986656
-132.989235037587 32.632563115390965
-32.11605132098799 -2.57110567742761e-13
7.332738889745223e-14 0
0 121.43120438476159
-115.12931750295782 27.80242375115498
-27.435051170792875 1.9760946278420368e-13
-9.422733028844873e-14 0
0 131.4283953322635
-124.03885057898883 33.85129327073272
-33.30106478971824 3.2964177984637165e-14
-3.079738556116914e-13 0
0 61.99775510506341
-60.35977838713657 27.26740802866568
-26.93308276815847 -1.1011070419062888e-13
3.4844005788868893e-13 0


c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\

0 82.62452246740499
-79.71175873198568 38.25252834413748
-37.58167163521154 -7.765718561452084e-14
-2.1305317863394438e-13 0
0 145.5004405507706
-136.4368358673931 23.588044787419065
-23.317230978203142 -3.1013920029561334e-13
1.6760468986135712e-13 0
0 154.07493071480692
-143.89714717951125 37.50764373266779
-36.81621994758793 -8.362823262078055e-14
5.29435730919955e-14 0
0 130.89328651042095
-123.56774202014522 21.192661318526024
-20.977267656591675 -3.840456557276608e-14
-6.537286755297166e-14 0
0 139.411228044116
-131.08838895364582 40.70143296505877
-39.899134545149636 -1.9962569363679396e-13
3.1314561995193173e-13 0
0 178.70258628301397
-164.98439005037721 18.48119512183826
-18.309106819252165 -1.1253230185071791e-13
-9.03784015002202e-14 0
0 204.8114416228094
-186.7071251558004 39.82280443378358
-38.99894177152789 1.6713371587382298e-13
-2.518309782593925e-13 0


...

c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\

0 209.18297338786252
-190.05053453930273 111.4045610509838
-104.79337708700176 -4.845576928009813e-14
5.240829771320202e-14 0
0 131.56047590726524
-124.13848972909257 64.7470437060163
-62.72631716083841 1.917621849733506e-14
-7.822607955508771e-15 0
0 109.9741746669394
-104.80185187063815 53.601329532938664
-52.24795185218306 -8.520944974509456e-14
1.615041339856288e-13 0
0 123.14862983730089
-116.6655116232856 32.30721597327019
-31.81020011586265 4.641754251711488e-14
9.150931561883049e-15 0
0 132.93796784339563
-125.37385493982364 41.84521446433707
-41.00262804673972 -2.350186297539491e-14
3.0642729334618744e-14 0


c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\

0 85.97569195139836
-82.8235094758835 19.25152413188129
-19.08120860530995 -2.9275782113481564e-13
9.380259562107285e-14 0
0 58.00204722138463
-56.56724869089828 43.89665877707311
-43.03243852942146 -6.339693183195585e-14
-1.0054646338087258e-13 0
0 61.99888341124492
-60.36134391134138 18.502569044571594
-18.34869453554278 -2.5480261747258694e-13
3.4721743380851345e-13 0


c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\get_powers.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_power_df = pd.concat([node_power_df, pd.DataFrame([[i, bus, P_node, Q_node,voltage_magnitudes]], columns=column)])
c:\Users\joao9\GitHub\opendss\src\powerflow\